In [ ]:
#!pip install pyarrow

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pyarrow as pa
import pyarrow.parquet as pq
import time

# too see max columns
pd.set_option('display.max_columns',None)

In [2]:
#churn_df = pd.read_parquet("../Data/churn.parquet")
churn_df = pd.read_csv("../Data/churn.csv")

In [3]:
churn_df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [4]:
churn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
churn_df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [6]:
#count of string value into the column.
count=0
for i in churn_df.TotalCharges:
    if i==' ':
        count+=1
print('count of empty string:- ',count)

count of empty string:-  11


In [7]:
# replace this empty string with nan
churn_df['TotalCharges'] = churn_df['TotalCharges'].replace(" ",np.nan)
# Change TotalCharges column to float
churn_df['TotalCharges'] = churn_df['TotalCharges'].astype(float)

In [8]:
# find null values
churn_df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [9]:
# replace null with mean
churn_df['TotalCharges'] = churn_df['TotalCharges'].fillna(churn_df['TotalCharges'].mean())

In [11]:
# Drop customerID column
churn_df.drop('customerID', axis=1, inplace=True)

In [ ]:
# replace yes and No categorical variables with Yes = 1 and No = 0
#for i in c:
#    churn_df[i] = churn_df[i].replace('Yes',1)
#    churn_df[i] = churn_df[i].replace('No',0)

In [ ]:
# replace male = 1 and female = 0
#churn_df.gender = churn_df.gender.replace('Male',1)
#churn_df.gender = churn_df.gender.replace('Female',0)

In [ ]:
#churn_df.MultipleLines.unique()

In [ ]:
# replace MultipleLines "No Phone Service" with 2
#churn_df.MultipleLines = churn_df.MultipleLines.replace('No phone service',2)

In [ ]:
#churn_df.InternetService.unique()

## Verify Numerical and Categorical Features

In [12]:
# Numerical Features
n = list(churn_df.select_dtypes(include=['int64','float64']).keys())
print(n)

['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']


In [13]:
# Categorical Variables
c = list(churn_df.select_dtypes(include='O').keys())
print(c)

['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']


In [14]:
# Save to csv
churn_df.to_csv('../Data/churn_cleaned.csv', index = False)

## Write Cleaned dataframe to parquet file

In [ ]:
# Convert DataFrame to Apache Arrow Table
#table = pa.Table.from_pandas(churn_df)

In [ ]:
# Parquet with GZIP compression and measure seconds to run
#start_time = time.time()
#pq.write_table(table, '../Data/churn_cleaned.parquet', compression='GZIP')
#print(time.time() - start_time, "seconds")